# Final Project

## Setup

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tqdm import tqdm

from IPython import display

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)

2023-12-07 15:32:30.656854: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 15:32:30.680814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 15:32:32.413541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-07 15:32:36.275042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [2]:
train_GAN = False
train_original_classifier = False
train_GAN_classifier = False

In [3]:
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()

In [4]:
train_images = train_images.reshape(train_images.shape[0], 32, 32, 3).astype('float32')
train_images = tf.image.resize(train_images, [64, 64])
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
# train_images = (train_images - 0.5) / 0.5

2023-12-07 15:32:36.985636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 15:32:36.985754: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 15:32:36.985794: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
BUFFER_SIZE = 60000
BATCH_SIZE = 128
noise_dim = 100

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## DCGAN Network

In [7]:
def make_generator_model(image_widthheight):
    initializer = tf.keras.initializers.RandomNormal(mean=0, stddev=0.02)
    batch_initializer = tf.keras.initializers.RandomNormal(mean=1, stddev=0.02)
    zeror = tf.keras.initializers.Zeros()
    model = tf.keras.Sequential()
    # model.add(layers.Dense(image_widthheight//8*image_widthheight//8*image_widthheight*4, use_bias=False, input_shape=(100,)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.ReLU())

    # model.add(layers.Reshape((image_widthheight//8, image_widthheight//8, image_widthheight*4)))
    # assert model.output_shape == (None, image_widthheight//8, image_widthheight//8, image_widthheight*4)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(image_widthheight*8, (4, 4), strides=(1, 1), use_bias=False, input_shape=(1,1,100), kernel_initializer=initializer))
    # assert model.output_shape == (None, image_widthheight//8, image_widthheight//8, image_widthheight*4)
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(image_widthheight*4, (4, 4), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=initializer))
    # assert model.output_shape == (None, image_widthheight//4, image_widthheight//4, image_widthheight*2)
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(image_widthheight*2, (4, 4), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=initializer))
    # assert model.output_shape == (None, image_widthheight//4, image_widthheight//4, image_widthheight*2)
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(image_widthheight, (4, 4), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=initializer))
    # assert model.output_shape == (None, image_widthheight//2, image_widthheight//2, image_widthheight)
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=initializer, activation='tanh'))
    # assert model.output_shape == (None, image_widthheight, image_widthheight, 3)

    return model

In [8]:
def make_discriminator_model(image_widthheight):
    initializer = tf.keras.initializers.RandomNormal(mean=0, stddev=0.02)
    batch_initializer = tf.keras.initializers.RandomNormal(mean=1, stddev=0.02)
    zeror = tf.keras.initializers.Zeros()
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(image_widthheight, (4, 4), strides=(2, 2), padding='same', kernel_initializer=initializer,
                                     input_shape=[image_widthheight, image_widthheight, 3]))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2D(image_widthheight*2, (4, 4), strides=(2, 2), padding='same', kernel_initializer=initializer))
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2D(image_widthheight*4, (4, 4), strides=(2, 2), padding='same', kernel_initializer=initializer))
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2D(image_widthheight*8, (4, 4), strides=(2, 2), padding='same', kernel_initializer=initializer))
    model.add(layers.BatchNormalization(gamma_initializer=batch_initializer, beta_initializer=zeror))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2D(1, (4, 4), strides=(2, 2), kernel_initializer=initializer, activation="sigmoid"))

    return model

In [9]:
generator = make_generator_model(64)

generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose (Conv2DTr  (None, 4, 4, 512)         819200    
 anspose)                                                        
                                                                 
 batch_normalization (Batch  (None, 4, 4, 512)         2048      
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 4, 4, 512)         0         
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 8, 8, 256)         2097152   
 Transpose)                                                      
                                                                 
 batch_normalization_1 (Bat  (None, 8, 8, 256)         1024      
 chNormalization)                                       

/home/cbeam18/.local/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [10]:
discriminator = make_discriminator_model(64)

discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 batch_normalization_4 (Bat  (None, 16, 16, 128)       512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 256)         524544    
                                                      

In [11]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5, beta_2=0.999)

In [12]:
import matplotlib.pyplot as plt
from IPython import display

seed = tf.random.normal([49, 1, 1, noise_dim])

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(7, 7))

  for i in range(49):
      plt.subplot(7, 7, i+1)
      plt.imshow((predictions[i] + 1) / 2)
      plt.axis('off')

  plt.savefig('outputs/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()
  plt.close()

In [13]:
@tf.function
def train_step(images):
    noises = tf.random.normal([BATCH_SIZE, 1, 1, noise_dim])
    with tf.GradientTape() as disc_tape:
        generated_images = generator(noises, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        disc_loss = discriminator_loss(real_output, fake_output)

    with tf.GradientTape() as gen_tape:
        generated_images = generator(noises, training=True)
        fake_output = discriminator(generated_images, training=True)
        g_loss = generator_loss(fake_output)
        gen_loss = g_loss

    grad_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    grad_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(grad_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(grad_disc, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, epochs):
    resize_layer = tf.keras.layers.Resizing(64, 64)
    for epoch in tqdm(range(epochs)):
        # start = time.time()
        for batch in dataset:
            gen_loss, disc_loss = train_step(batch)
            # train_step(resize_layer(batch))

        print(f"Epoch {epoch+1}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}")
        generate_and_save_images(generator, epoch, seed)
        # print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [14]:
if train_GAN:
  train(train_dataset, 150)

In [15]:
if train_GAN:
  generator.save_weights("models/generator_64.ckpt")
  discriminator.save_weights("models/discriminator_64.ckpt")
else:
  generator.load_weights("models/generator_64.ckpt")
  discriminator.load_weights("models/discriminator_64.ckpt")

## MobileNet V2 - No GAN inputs

In [16]:
keras.backend.clear_session()
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()
train_images = train_images / 255
train_images = tf.image.resize(train_images, (64, 64))

In [17]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomFlip("vertical"),
        keras.layers.RandomZoom(height_factor=0.1,width_factor=0.1)
    ]
)

In [18]:
base_mobile_model = keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(128,128,3),
    include_top=False
)

base_mobile_model.trainable = False

inputs = keras.Input(shape=(64,64,3))
x = data_augmentation(inputs)

scaled_layer = keras.layers.Rescaling(scale=1/255.0)
resize_layer = tf.keras.layers.Resizing(128, 128)
x = resize_layer(x)

x = base_mobile_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(10)(x)
mobile_model = keras.Model(inputs, outputs)

mobile_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 64, 64, 3)         0         
                                                                 
 resizing (Resizing)         (None, 128, 128, 3)       0         
                                                                 
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0     

In [19]:
if train_original_classifier:
  keras.backend.clear_session()
  mobile_model.compile(
      optimizer=keras.optimizers.Adam(),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'],
  )

  # logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  epochs = 20
  mobile_model.fit(train_images, train_labels, batch_size=128, epochs=epochs) #, callbacks=[tensorboard_callback])

In [20]:
if train_original_classifier:
  keras.backend.clear_session()
  base_mobile_model.trainable = True

  mobile_model.compile(
      optimizer=keras.optimizers.Adam(1e-5),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'],
  )

  mobile_model.fit(train_images, train_labels, batch_size=128, epochs=10) #, callbacks=[tensorboard_callback])

In [21]:
if train_original_classifier:
  mobile_model.save_weights("models/mobile_model_64.ckpt")
else:
  mobile_model.load_weights("models/mobile_model_64.ckpt")

## Generate Fake Dataset

In [22]:
holder_0 = []
holder_1 = []
holder_2 = []
holder_3 = []
holder_4 = []
holder_5 = []
holder_6 = []
holder_7 = []
holder_8 = []
holder_9 = []

holder_holder = [holder_0, holder_1, holder_2, holder_3, holder_4, holder_5, holder_6, holder_7, holder_8, holder_9]

full_holder = [False, False, False, False, False, False, False, False, False, False]

full_num = 0

In [23]:
gen_number = 100
size_per_class = 1000
for _ in range(1000):
    # Break when done generating images
    if all(full_holder):
        break
    # Generate Images
    seed = tf.random.normal([gen_number, 1, 1, noise_dim])
    gen_images = generator(seed, training=False)

    # Get Labels
    preds = mobile_model(gen_images * 0.5 + 0.5, training=False)
    labels = tf.reshape(tf.cast(tf.math.argmax(preds, axis=1), dtype=tf.uint8), (gen_number, 1))

    # Sort labeled images into respective holder if not full
    for idx in range(gen_number):
        label = labels[idx][0].numpy()
        if not full_holder[label]:
            holder_holder[label].append(tf.cast(gen_images[idx] * 127.5 + 127.5, "uint8"))
            if len(holder_holder[label]) == size_per_class:
                full_holder[label] = True


2023-12-07 15:32:40.697534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-07 15:32:40.708338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700


In [25]:
for classIdx in range(10):
    for imageIdx in range(size_per_class):
        file_path = "fake_dataset2/%d/%d_%d.png" % (classIdx, classIdx, imageIdx)
        tf.keras.utils.save_img(file_path, holder_holder[classIdx][imageIdx])